In [40]:
import pandas as pd
from scipy.spatial import distance
import zipfile
import os.path
import os
from scipy.stats import rankdata
import operator

In [11]:
df_book = pd.read_csv("book-vec-group.csv")
df_book

,Unnamed: 0,Book-Title,Book-Author_x,Book-Summary,ISBN,group,BERT,word2vec
0,0,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...",345316413X,23,"[-0.29636892676353455, 0.8470680117607117, 0.7...",[ 4.51489369e-02 6.01137093e-02 -1.26928838e-...
1,1,The Plague,Albert Camus,The text of The Plague is divided into five p...,0075536498,47,"[0.2956414222717285, 1.319756031036377, 0.3554...",[ 0.05818041 0.06845398 0.00783395 0.036492...
2,2,An Enquiry Concerning Human Understanding,David Hume,The argument of the Enquiry proceeds by a ser...,0872202291,5,"[1.2887966632843018, 1.2676935195922852, 1.117...",[ 0.06308864 0.02407891 0.03377491 0.085396...
3,3,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...",0449213943,8,"[0.2628381550312042, 1.452850103378296, 1.0604...",[ 0.08981435 0.08089734 0.02092221 0.061505...
4,4,A Wizard of Earthsea,Ursula K. Le Guin,"Ged is a young boy on Gont, one of the larger...",0689317204,32,"[-0.12000640481710434, 1.7298859357833862, 0.7...",[ 0.07931867 0.07550479 -0.00114273 0.017464...
...,...,...,...,...,...,...,...,...
5845,5845,Gates of Paradise,V. C. Andrews,"A novel about Annie Stonewall, the daughter o...",0671670646,14,"[0.37652456760406494, 1.5239266157150269, 0.91...",[ 0.02671554 -0.01298588 -0.03296045 0.033805...
5846,5846,Shelter,Harlan Coben,"After Mickey Bolitar moves in with his uncle,...",0385313896,11,"[-0.22981029748916626, 1.2231945991516113, 1.8...",[ 0.07016824 0.04757968 -0.036401 0.025883...
5847,5847,Remote Control,Andy McNab,The series follows the character of Nick Ston...,0345428056,43,"[0.23090682923793793, 0.9125347137451172, 0.59...",[ 0.04248047 0.07196384 -0.0039071 0.050720...
5848,5848,The Simpsons: A Complete Guide to Our Favorite...,Matt Groening,"{| class=""wikitable"" |- !Seasons covered !Boo...",0060952520,3,"[0.0948098748922348, 1.4162523746490479, 0.936...",[-2.14607932e-02 -4.20726429e-02 -3.74117765e-...


In [13]:
def recommend_on_group(book_title):
    group = df_book[df_book["Book-Title"] == book_title]["group"].tolist()[0]
    recommend_list = df_book[df_book["group"] == group]["Book-Title"].tolist()
    return recommend_list

recommend_on_group('Remote Control')

['A Scanner Darkly',
 'The A.B.C. Murders',
 'Eight Skilled Gentlemen',
 'The Redemption of Althalus',
 'Firewall',
 'Patriot Games',
 'Never Dream of Dying',
 'The Sands of Time',
 'Country of the Blind',
 'The Great Train Robbery',
 'Filth',
 'Nylon Angel',
 'Strip Tease',
 'Detective',
 'The Deceivers',
 'The Janson Directive',
 'The Big Nowhere',
 'The Man',
 'Point of Impact',
 'Revenant',
 'Sidetracked',
 'Adam and Eve and Pinch Me',
 'The Face',
 'The Last Days of Louisiana Red',
 'Hideaway',
 'An Act of Terror',
 'Teckla',
 'Jhereg',
 'Yendi',
 'Night Frost',
 'Andersonville',
 'Winter Moon',
 'Hostage',
 'The Hanging Garden',
 'Warlock',
 'To Play the Fool',
 'After the First Death',
 'The Twenty-Seventh City',
 'Bel Canto',
 'Circus',
 'Dance Hall of the Dead',
 'The Judas Pair',
 'Listening Woman',
 'The Revengers',
 'The Annihilators',
 'The Demolishers',
 'Still Life with Crows',
 'The Damnation Game',
 "Hornet's Nest",
 'Sacred Clowns',
 'Murder at the Savoy',
 'Four Blin

In [14]:
def strtoVecBERT(str):
    str = str[1:len(str) - 1]
    list_str = list(str.split(","))
    list_float = []
    for element in list_str:
        list_float.append(float(element))
    return list_float

def strtoVecword2vec(str):
    str = str[1:len(str) - 1]
    list_str = list(str.split(" "))
    list_float = []
    for element in list_str:
        if len(element) > 0:
            list_float.append(float(element))
    return list_float
 
# Driver code
str1 = df_book[df_book["Book-Title"] == "A Clockwork Orange"]["BERT"].tolist()[0]
strtoVecBERT(str1)

[-0.29636892676353455,
 0.8470680117607117,
 0.7940486669540405,
 0.21900121867656708,
 0.6064422130584717,
 -0.12001042813062668,
 1.759630799293518,
 0.4376492500305176,
 0.7948243618011475,
 0.5240957140922546,
 0.4789082109928131,
 0.8138662576675415,
 0.8186842799186707,
 0.2532104551792145,
 0.16700415313243866,
 0.6193603277206421,
 1.3077552318572998,
 0.418896347284317,
 0.584417462348938,
 1.4660550355911255,
 0.24281956255435944,
 0.30600589513778687,
 1.207471251487732,
 0.30437761545181274,
 1.5898282527923584,
 1.3282071352005005,
 0.8541168570518494,
 0.9618334174156189,
 -0.3068588674068451,
 0.5460641980171204,
 0.8541282415390015,
 1.6318871974945068,
 0.4036577641963959,
 0.5943519473075867,
 0.6907811760902405,
 1.217929720878601,
 1.022335171699524,
 0.8068059682846069,
 0.997705340385437,
 0.7882225513458252,
 1.3315398693084717,
 0.2419622242450714,
 -0.34073203802108765,
 0.0403386615216732,
 -0.1250886470079422,
 0.05981064960360527,
 0.41403263807296753,
 1.23

In [15]:
def recommend_one_on_BERT(book_title):
    vec1 = strtoVecBERT(df_book[df_book["Book-Title"] == book_title]["BERT"].tolist()[0])
    min_distance = 10000
    most_similar_book = ''
    for index, row in df_book.iterrows():
        vec2 = strtoVecBERT(row["BERT"])
        dist = distance.euclidean(vec1, vec2) 
        if(dist < min_distance and dist != 0):
            min_distance = dist
            most_similar_book = row["Book-Title"]

    return most_similar_book

recommend_one_on_BERT("Firewall")

'Sidetracked'

In [16]:
def recommend_one_on_word2vec(book_title):
    vec1 = strtoVecword2vec(df_book[df_book["Book-Title"] == book_title]["word2vec"].tolist()[0])
    min_distance = 10000
    most_similar_book = ''
    for index, row in df_book.iterrows():
        vec2 = strtoVecword2vec(row["word2vec"])
        dist = distance.euclidean(vec1, vec2) 
        if(dist < min_distance and dist != 0):
            min_distance = dist
            most_similar_book = row["Book-Title"]

    return most_similar_book

recommend_one_on_word2vec("Firewall")

'The Man Who Never Missed'

In [51]:
def recommend_n_on_ensemble(book_title, n):
    word2vec_list = []
    vec1 = strtoVecword2vec(df_book[df_book["Book-Title"] == book_title]["word2vec"].tolist()[0])
    for index, row in df_book.iterrows():
        vec2 = strtoVecword2vec(row["word2vec"])
        dist = distance.euclidean(vec1, vec2) 
        title2 = row["Book-Title"]
        word2vec_list.append((title2, dist))
    
    word2vec_sorted = sorted(word2vec_list, key=lambda tup: tup[1])[1:len(word2vec_list)]
    word2vec_dist = [x[1] for x in word2vec_sorted]
    word2vec_rank = rankdata(word2vec_dist)

    bert_list = []
    vec1 = strtoVecBERT(df_book[df_book["Book-Title"] == book_title]["BERT"].tolist()[0])
    for index, row in df_book.iterrows():
        vec2 = strtoVecBERT(row["BERT"])
        dist = distance.euclidean(vec1, vec2) 
        title2 = row["Book-Title"]
        bert_list.append((title2, dist))

    bert_sorted = sorted(bert_list, key=lambda tup: tup[1])[1:len(bert_list)]
    bert_dist = [x[1] for x in bert_sorted]
    bert_name = [x[0] for x in bert_sorted]
    bert_rank = rankdata(bert_dist)

    scores = []
    for i in range(len(word2vec_rank)):
       score = 1/ (i + 1)  #rank of word2vec; add 1 to avoid division by 0
       index_bert_name = bert_name.index(word2vec_sorted[i][0])
       rank = bert_rank[index_bert_name]
       score = score + 1 / (rank + 1) 
       scores.append((word2vec_sorted[i][0], score))
    
    scores_sorted = sorted(scores, key=lambda tup: tup[1])[:n]
    book_list = [x[0] for x in scores_sorted]
    return book_list

book_list = recommend_n_on_ensemble("Firewall", 10)
book_list

['Flame',
 'The Eyes of Darkness',
 'Butterfly',
 'Term Limits',
 'The Saint and the Hapsburg Necklace',
 'Grania: She-King of the Irish Seas',
 'Irish Linen',
 'Rasputin',
 'Irish Stew!',
 'The Apprentice']